# Initialize

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

//jupyter notebook way to create spark session
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

val sc = spark.sparkContext

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

//jupyter notebook way to create spark session

spark: SparkSession = org.apache.spark.sql.SparkSession@593b72
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@18611a0

# Load text files

In [16]:
val file_contents = sc.wholeTextFiles("/home/pi/sparktest/data/invertedIndex")

file_contents: org.apache.spark.rdd.RDD[(String, String)] = /home/pi/sparktest/data/invertedIndex MapPartitionsRDD[23] at wholeTextFiles at cmd15.sc:1

In [18]:
val file_contents_refined = file_contents.map{
case(id,text)=>
    val lastSep = id.lastIndexOf("/")
    val id2 = if(lastSep <0) id.trim
        else id.substring(lastSep+1,id.length).trim
    val text2 = text.trim.replaceAll("""\s*\n\s*"""," ")
(id2,text2)
}

// file_contents_refined.map(k=>k._1).collect
file_contents_refined.saveAsTextFile("/home/pi/sparktest/data/invertedIndex/result")

: 

In [19]:
val lineRE = """^\s*\(([^,]+),(.*)\)\s*$""".r
val rawInput = sc.textFile("/home/pi/sparktest/data/invertedIndex/result")
val input = rawInput.map{
    case lineRE(name,text)=>(name.trim,text.toLowerCase)
    case badLine =>
    ("","")
}
//input.collect()
// val input = sc.textFile("/home/pi/sparktest/data/invertedIndex/result")
// input.collect()

lineRE: scala.util.matching.Regex = ^\s*\(([^,]+),(.*)\)\s*$
rawInput: org.apache.spark.rdd.RDD[String] = /home/pi/sparktest/data/invertedIndex/result MapPartitionsRDD[29] at textFile at cmd18.sc:2
input: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[30] at map at cmd18.sc:3

In [40]:
val rawWordContMap = input.flatMap{
    case(path,text) =>
        text.trim.split("""[^\w']""").map{
            word=>((word,path),1)
        }
}

val stopWords = Set("to","all","the","sql","slow","with","all","disk")

val rawWordContMapClean = rawWordContMap.filter{
    case((word,_),_) => stopWords.contains(word) == false
}
rawWordContMapClean.take(3)

val finalMap = rawWordContMapClean.reduceByKey((count1,count2)=>count1+count2)
finalMap.take(20)

// val rawWordContMap = input.flatMap(t=>t._2.split("""[^\w']"""))
// rawWordContMap.take(10)

take at cmd39.sc:13

1 / 1

filter at cmd39.sc:10

3 / 3

take at cmd39.sc:16

1 / 1

take at cmd39.sc:16

2 / 2

rawWordContMap: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[63] at flatMap at cmd39.sc:1
stopWords: Set[String] = Set("sql", "slow", "disk", "all", "to", "with", "the")
rawWordContMapClean: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[64] at filter at cmd39.sc:10
res39_3: Array[((String, String), Int)] = Array(
  (("using", "webpage1.txt"), 1),
  (("spark", "webpage1.txt"), 1),
  (("spark", "webpage1.txt"), 1)
)
finalMap: org.apache.spark.rdd.RDD[((String, String), Int)] = ShuffledRDD[65] at reduceByKey at cmd39.sc:15
res39_5: Array[((String, String), Int)] = Array(
  (("compare", "webpage2.txt"), 1),
  (("using", "webpage1.txt"), 1),
  (("sparkcontext", "webpage1.txt"), 1),
  (("hadoop", "webpage1.txt"), 2),
  (("spark", "webpage1.txt"), 5),
  (("processing", "webpage2.txt"), 1),
  (("dataframe", "webpage1.txt"), 1),
  (("in", "webpage2.txt"), 1),
  (("data", "webpage2.txt"), 1),
  (("process", "webpage2.txt"), 1),
  (("spark", "webpage

In [33]:
val moreGoodFinalMap = finalMap.map{
    case((word,path),n) => (word,(path,n))
}
.groupByKey

moreGoodFinalMap.take(3)

val superGoodFinalMap = moreGoodFinalMap
.map{
    case(word, iterable) => (word,iterable.mkString(","))
}

superGoodFinalMap.take(3)

map at cmd32.sc:1

3 / 3

take at cmd32.sc:6

1 / 1

take at cmd32.sc:13

1 / 1

moreGoodFinalMap: org.apache.spark.rdd.RDD[(String, Iterable[(String, Int)])] = ShuffledRDD[57] at groupByKey at cmd32.sc:1
res32_1: Array[(String, Iterable[(String, Int)])] = Array(
  ("slow", CompactBuffer(("webpage2.txt", 1))),
  ("with", CompactBuffer(("webpage2.txt", 1))),
  ("", CompactBuffer(("webpage2.txt", 1)))
)
superGoodFinalMap: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[58] at map at cmd32.sc:9
res32_3: Array[(String, String)] = Array(
  ("slow", "(webpage2.txt,1)"),
  ("with", "(webpage2.txt,1)"),
  ("", "(webpage2.txt,1)")
)

# refined

In [37]:
val superGoodFinalMapSorted = moreGoodFinalMap
.map{
    case(word, iterable) => 
        val vect = iterable.to[Vector].sortBy{
            case (path,n) => (-n,path)
        }
        (word,vect.mkString(", "))
}

superGoodFinalMapSorted.collect()

collect at cmd36.sc:10

3 / 3

superGoodFinalMapSorted: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[62] at map at cmd36.sc:2
res36_1: Array[(String, String)] = Array(
  ("slow", "(webpage2.txt,1)"),
  ("with", "(webpage2.txt,1)"),
  ("", "(webpage2.txt,1)"),
  ("sql", "(webpage1.txt,1)"),
  ("use", "(webpage2.txt,1)"),
  ("the", "(webpage2.txt,1)"),
  ("dataframe", "(webpage1.txt,1)"),
  ("hadoop", "(webpage2.txt,3), (webpage1.txt,2)"),
  ("memory", "(webpage2.txt,1)"),
  ("do", "(webpage2.txt,1)"),
  ("disk", "(webpage2.txt,1)"),
  ("all", "(webpage2.txt,1)"),
  ("processing", "(webpage2.txt,1)"),
  ("using", "(webpage1.txt,1)"),
  ("compare", "(webpage2.txt,1)"),
  ("spark", "(webpage1.txt,5), (webpage2.txt,2)"),
  ("process", "(webpage2.txt,1)"),
  ("to", "(webpage2.txt,1)"),
  ("data", "(webpage2.txt,1)"),
  ("in", "(webpage2.txt,1)"),
  ("sparkcontext", "(webpage1.txt,1)")
)